# TSMixer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import time
from random import randrange
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import StandardScaler

# Loading Data

In [ ]:
    filepath = ('SP100_vol (original).xlsx')

    df_raw = pd.read_excel(filepath)
    df = df_raw.set_index('Date')

    # split train/valid/test
    n = len(df)
    train_end = int(n * 0.7)-5
    val_end = int(n * 0.7)+5
    test_end = n

test_true=df[val_end - 21 : test_end]
test_true=test_true.iloc[21:]

# Test

In [ ]:
class DataLoader:

  def __init__(self, batch_size, seq_len, pred_len):
    self.batch_size = batch_size
    self.seq_len = seq_len
    self.pred_len = pred_len
    self.target_slice = slice(0, None)

    self._read_data()

  def _read_data(self):

    filepath = ('SP100_vol (original).xlsx')

    df_raw = pd.read_excel(filepath)
    df = df_raw.set_index('Date')

    # split train/valid/test
    n = len(df)
    train_end = int(n * 0.7)-5
    val_end = int(n * 0.7)+5
    test_end = n

    train_df = df[int(train_end*0.5):train_end]
    val_df = df[train_end - self.seq_len : val_end]
    test_df = df[val_end - self.seq_len : test_end]

    # standardize by training set
    self.scaler = StandardScaler()
    self.scaler.fit(train_df.values)

    def scale_df(df, scaler):
      data = scaler.transform(df.values)
      return pd.DataFrame(data, index=df.index, columns=df.columns)

    self.train_df = scale_df(train_df, self.scaler)
    self.val_df = scale_df(val_df, self.scaler)
    self.test_df = scale_df(test_df, self.scaler)
    self.n_feature = self.train_df.shape[-1]

  def _split_window(self, data):
    inputs = data[:, : self.seq_len, :]
    labels = data[:, self.seq_len :, self.target_slice]

    inputs.set_shape([None, self.seq_len, None])
    labels.set_shape([None, self.pred_len, None])
    return inputs, labels

  def _make_dataset(self, data, shuffle=False):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=(self.seq_len + self.pred_len),
        sequence_stride=1,
        shuffle=shuffle,
        batch_size=self.batch_size,
    )
    ds = ds.map(self._split_window)
    return ds

  def inverse_transform(self, data):
    return self.scaler.inverse_transform(data)

  def get_train(self, shuffle=True):
    return self._make_dataset(self.train_df, shuffle=shuffle)

  def get_val(self):
    return self._make_dataset(self.val_df, shuffle=False)

  def get_test(self):
    return self._make_dataset(self.test_df, shuffle=False)

  def make_forecast(self, model):
    test_data = self.test_df.values
    predictions = []

    for i in range(0, len(test_data) - self.seq_len, self.pred_len):
        input_seq = test_data[i:i + self.seq_len]
        input_seq = input_seq.reshape((1, self.seq_len, -1))

            # Make prediction and reshape output
        pred = model.predict(input_seq)
        pred = pred.reshape(self.pred_len, -1)

        predictions.append(pred)

        # Concatenate all predictions
    predictions = np.concatenate(predictions, axis=0)
    return predictions

In [ ]:
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(89102543)

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  #early_stop_callback = tf.keras.callbacks.EarlyStopping(
  #    monitor='val_loss',
  #    patience=140
  #)

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 168 #accounting for early stop
      #validation_data=val_data,
     # callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')

Epoch 1/168
37/37 [==============================] - 8s 11ms/step - loss: 20.5936 - mae: 3.5110 - mean_absolute_percentage_error: 3390.1360 - mean_squared_logarithmic_error: 0.8920
Epoch 2/168
37/37 [==============================] - 0s 11ms/step - loss: 13.9422 - mae: 2.8963 - mean_absolute_percentage_error: 2468.4941 - mean_squared_logarithmic_error: 0.7443
Epoch 3/168
37/37 [==============================] - 0s 11ms/step - loss: 10.4021 - mae: 2.5086 - mean_absolute_percentage_error: 2466.1965 - mean_squared_logarithmic_error: 0.6508
Epoch 4/168
37/37 [==============================] - 0s 12ms/step - loss: 8.1692 - mae: 2.2276 - mean_absolute_percentage_error: 2022.4331 - mean_squared_logarithmic_error: 0.5779
Epoch 5/168
37/37 [==============================] - 0s 12ms/step - loss: 6.6204 - mae: 1.9952 - mean_absolute_percentage_error: 1857.2306 - mean_squared_logarithmic_error: 0.5078
Epoch 6/168
37/37 [==============================] - 0s 11ms/step - loss: 5.5558 - mae: 1.8241 - 

In [ ]:
predictions = model.predict(test_data)
scaled_preds = predictions
scaled_preds=scaled_preds.reshape(scaled_preds.shape[0], scaled_preds.shape[2])
scaled_preds.shape
preds = data_loader.inverse_transform(scaled_preds)
RMSE_test=np.sqrt(((test_true.iloc[:]-preds)**2).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))
MAE_test=((abs(test_true.iloc[:]-preds)).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))
MAPE_test=((abs(test_true.iloc[:]-preds)/test_true.iloc[:]).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))
QLIKE_test=((test_true.iloc[:]/preds)-np.log(test_true.iloc[:]/preds)-1).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))
print(RMSE_test)
print(MAE_test)
print(MAPE_test)
print(QLIKE_test)

32/32 [==============================] - 1s 8ms/step
0.0037723119197032875
0.002407456392479064
0.19208954870698916
0.03102961586667518


In [ ]:
test_true.iloc[:]=preds
test_true.to_excel("TSMixer forecasts.xlsx")